<img src="Figures/top_ML.png" alt="Drawing" style="width: 1000px;"/>

# "Digitalización de una carga doméstica"

## Postítulo "Digital Energy", Máster Smart Energy

## Módulo: Digitalización de la Energía

## Alumno: Max Antonio Missene Montecinos

### Objetivo:

#### Monitoreo de una carga doméstica mediante un procesador de datos construido en base a un microcontrolador Arduino.

### Pautas generales:

1. Descripción de la carga seleccionada para nuestro Proyecto. Modelo, potencia nominal, uso habitual que hacéis de ella, etc.
2. Descripción de vuestro montaje: Fotos del montaje con la carga, explicación breve del código para la realización de la medición.
3. Análisis preliminar de consumo de la carga: Captura del a corriente instantánea, grabar datos de corriente en RMS y realizar alguna gráfica (Excel mismo) y comentar el consumo observado.

## Generales

#### El primer paso que se realizará en el desarrollo del presente trabajo, será importar las librerías y funciones de librerías que utilizaremos a lo largo del código.

In [1]:
# Librerías usadas

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
from datetime import date, time, datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

#### A continuación, se procederá a importar la base de datos.

In [2]:
# Generar DataFrame (DF) a partir de base de datos de excel
columns=['Fecha','Corriente','Potencia']
BD_carga = pd.read_csv('Data/data_700_F_20h.csv', names=columns)

### 1. Entendimiento de los datos, estadística descriptiva y visualizaciones

#### En primera instancia, se determinará el número de dimensiones que posee la base de datos.

In [3]:
# Se procederá a conocer las dimensiones del DF con la función "shape".
BD_carga.shape

(223, 3)

#### Es posible notar que el DataFrame contine 3 columnas y 223 filas. 

#### Así, se desplegaran las primeras filas del dataframe, para examinar de forma preliminar los datos.

In [4]:
# Desplegar primeras 5 filas del DF a partir de la función "head".
BD_carga.head()

,Fecha,Corriente,Potencia
0,2022-06-07 20:47:44.244684,0.21,48.75
1,2022-06-07 20:47:45.248650,0.21,49.23
2,2022-06-07 20:47:46.251581,0.22,49.77
3,2022-06-07 20:47:47.254675,0.21,47.84
4,2022-06-07 20:47:48.254259,0.20,46.90


#### De manera análoga, se desplegaran las primeras últimas filas del dataframe, para examinar de forma preliminar los datos.

In [5]:
# Desplegar última 5 filas del DF a partir de la función "tail".
BD_carga.tail()

,Fecha,Corriente,Potencia
218,2022-06-07 20:51:22.734827,0.22,51.05
219,2022-06-07 20:51:23.738835,0.22,49.72
220,2022-06-07 20:51:24.737518,0.21,48.42
221,2022-06-07 20:51:25.741341,0.22,49.63
222,2022-06-07 20:51:26.740842,0.21,49.19


In [6]:
# Desplegar última 5 filas del DF a partir de la función "tail".
BD_carga.dtypes

Fecha         object
Corriente    float64
Potencia     float64
dtype: object

#### Ya pudiendo mirar el contenido del dataframe de manera general, se realizará el cálculo de valores estadísticos de sus columnas.

In [7]:
# Desplegar última 5 filas del DF a partir de la función "tail".
BD_carga['Fecha']=pd.to_datetime(BD_carga['Fecha'])
BD_carga.dtypes


Fecha        datetime64[ns]
Corriente           float64
Potencia            float64
dtype: object

In [8]:
# Se aplica la función "describe" al dataframe, el cual calculará algunos parámetros estadísticos a cada columna.
BD_carga.describe()

,Corriente,Potencia
count,223.000000,223.000000
mean,6.099327,1402.838161
std,1.320787,303.770429
min,0.200000,46.900000
25%,6.390000,1468.585000
50%,6.400000,1472.510000
75%,6.420000,1475.975000
max,6.560000,1509.800000


In [9]:
BD_carga.isna().sum()

Fecha        0
Corriente    0
Potencia     0
dtype: int64

In [11]:
BD_carga['Energy_kWh']=0.0
BD_carga['time_dif']=0.0
BD_carga['cum_energy_kwh']=0.0
longitud=len(BD_carga['Energy_kWh'])

for x in range(0, longitud):
 if BD_carga['Corriente'][x]<0.0:
        BD_carga.at[x, 'Corriente']=0.0
 if BD_carga['Potencia'][x]<0.0:
        BD_carga.at[x, 'Potencia']=0.0
for x in range(1, longitud):
    BD_carga.at[x, 'Energy_kWh']=BD_carga['Potencia'][x]*((BD_carga['Fecha'][x]-BD_carga['Fecha'][x-1]).total_seconds()/3600000)
    BD_carga.at[x, 'time_dif']=(BD_carga['Fecha'][x]-BD_carga['Fecha'][x-1]).total_seconds()
    BD_carga.at[x, 'cum_energy_kwh']=BD_carga['cum_energy_kwh'][x-1]+BD_carga['Energy_kWh'][x]
    

In [12]:
BD_carga.head()

,Fecha,Corriente,Potencia,Energy_kWh,time_dif,cum_energy_kwh
0,2022-06-07 20:47:44.244684,0.21,48.75,0.000000,0.000000,0.000000
1,2022-06-07 20:47:45.248650,0.21,49.23,0.000014,1.003966,0.000014
2,2022-06-07 20:47:46.251581,0.22,49.77,0.000014,1.002931,0.000028
3,2022-06-07 20:47:47.254675,0.21,47.84,0.000013,1.003094,0.000041
4,2022-06-07 20:47:48.254259,0.20,46.90,0.000013,0.999584,0.000054


In [13]:
BD_carga.tail()

,Fecha,Corriente,Potencia,Energy_kWh,time_dif,cum_energy_kwh
218,2022-06-07 20:51:22.734827,0.22,51.05,0.000014,1.002421,0.087025
219,2022-06-07 20:51:23.738835,0.22,49.72,0.000014,1.004008,0.087039
220,2022-06-07 20:51:24.737518,0.21,48.42,0.000013,0.998683,0.087052
221,2022-06-07 20:51:25.741341,0.22,49.63,0.000014,1.003823,0.087066
222,2022-06-07 20:51:26.740842,0.21,49.19,0.000014,0.999501,0.087079


In [14]:
BD_carga['Energy_kWh'].sum()

0.08707945242375278

In [15]:
((BD_carga['time_dif'].sum())/60)/60

0.06180448833333333

In [18]:
# conda install -c plotly plotly
import plotly.offline as py;
from plotly import tools;
import plotly.graph_objs as go;
from ipywidgets import widgets;
from ipywidgets import interact;
import warnings
warnings.filterwarnings('ignore')
py.init_notebook_mode(connected= True)

In [19]:
fig = go.Figure(
    data=[
        go.Line(x=BD_carga['Fecha'][0:12*75],
                y=BD_carga['Potencia'][0:12*75], name="Power")
    ],
    layout=go.Layout(
        title=dict(text='Potencia RMS Hervidor Eléctrico: 20h;AguaFría;700ml'),
        xaxis=dict(title='Hora'),
        yaxis=dict(title='Potencia [W]')
    )
)
fig.show()
        
        

C:\Users\mmissene\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


